# 5. Value Betting Calculator

In this notebook we'll make use of our match dataset and the Logistc Regression model we've built in the last stage, to predict the chances for each player to win in any given match.

We'll then compare these chances with the odds available for those matches, and build a Dataframe where each row will contain information regarding which player we would have to bet on!



trying to simulate bets on matches where we think to have an edge on the betting market (i.e. a value bet occurs when the real odds for an event are higher than the odds "predicted" by our model).

Finally, we'll compare several betting strategies and we'll find out if it's possible to make a good ROI backtesting our betting strategies.

Let's import the libraries needed for this notebook:

In [1]:
import pandas as pd
from math import exp
import pickle

Now, we'll load the Logistc Regression model trained in last notebook:

In [2]:
lreg = pickle.load(open("logistic_regression.lr", 'rb'))

Let's take a look at the coefficients and the intercept of this model:

In [3]:
lreg.coef_

array([[-1.73864599e-02, -3.81198666e-03, -9.36402414e-03,
        -5.97945669e-04,  1.50978346e-01, -1.52837340e-01,
        -3.57503498e-02,  4.47955938e-02,  1.00935909e-01,
         7.25103567e-02, -2.22920261e-01, -1.49418394e-01,
         1.66817880e-01,  3.32629956e-03,  3.10259424e-02,
         4.12037575e-03,  8.05517053e-02,  5.41611951e-02,
        -5.04474133e-02, -1.07672188e-01, -1.72534801e-02,
        -1.74139355e-03, -2.71984766e-02,  5.69399319e-02,
        -1.12272012e-01,  0.00000000e+00,  2.16046639e-01,
         1.41095422e-02,  0.00000000e+00, -1.81537736e-03,
        -1.78056578e-02,  2.63484639e-02,  2.38633201e-01,
        -7.73646960e-02,  3.24647093e-02, -7.34292063e-02,
         1.60885598e-02,  0.00000000e+00, -6.71552601e-02,
        -2.59926432e-02, -5.08491370e-04, -6.95521772e-02,
         1.45563464e-01, -3.97128759e-02, -1.12358810e-02,
        -1.27421732e-01,  0.00000000e+00,  1.21102923e-01,
         1.07210735e-01,  8.00524174e-02,  0.00000000e+0

In [4]:
lreg.intercept_

array([-0.0447461])

Now, let's open the Coefficients dataframe saved in the last notebook, and let's create a dictionary object based on the dataframe, where the key is going to be the attribute name and the value will be the attribute's coefficient:

In [5]:
coefs_df = pd.read_csv("csv/Coefficients.csv")

In [6]:
coefs_df.head()

,Column,Coef
0,Acapulco,-0.017386
1,Adelaide,-0.003812
2,Amersfoort,-0.009364
3,Amsterdam,-0.000598
4,Atlanta,0.150978


In [7]:
coefs_map = {}
for ix, row in coefs_df.iterrows():

    attribute = row['Column']
    attribute_coef = row['Coef']

    coefs_map[attribute] = attribute_coef

In [8]:
coefs_map

{'Acapulco': -0.017386459891201182,
 'Adelaide': -0.0038119866637319937,
 'Amersfoort': -0.009364024137787877,
 'Amsterdam': -0.0005979456689940237,
 'Atlanta': 0.15097834603719484,
 'Auckland': -0.1528373404489276,
 'Bangkok': -0.035750349770572724,
 'Barcelona': 0.04479559380327373,
 'Basel': 0.10093590926652328,
 'Bastad': 0.0725103566687114,
 'Beijing': -0.2229202606153204,
 'Belgrade': -0.1494183943253299,
 'Bogota': 0.16681788040336756,
 'Brighton': 0.003326299557031982,
 'Brisbane': 0.0310259424224191,
 'Bucharest': 0.004120375753177304,
 'Buenos Aires': 0.08055170532536904,
 'Casablanca': 0.05416119507213096,
 'Chennai': -0.05044741331807452,
 'Cincinnati': -0.10767218807340748,
 'Copenhagen': -0.017253480079926053,
 'Costa Do Sauipe': -0.0017413935461831516,
 'Delray Beach': -0.02719847656586563,
 'Doha': 0.056939931850891225,
 'Dubai': -0.11227201208401627,
 'Dusseldorf': 0.0,
 'Eastbourne': 0.2160466386078776,
 'Estoril': 0.014109542196478296,
 'Geneva': 0.0,
 'Gstaad': -0.0

Let's load our matches Dataset

In [9]:
df = pd.read_csv("csv/FeatureCalculated_Data.csv")

Now our goal is to build another Dataframe, containing the informations about the the players, who wins each match, the average and maximum odds available for each player, and we'll use the logistic regression coefficients (and the intercept) to calculate the predicted chances to win for each player:

In [10]:
betting_data_df = df[(df["Avg_Pl0"] > 1) & (df["Avg_Pl1"] > 1) & (df["Max_Pl0"] > 1) & (df["Max_Pl1"] > 1)]

betting_df_list = []
for ix, irow in betting_data_df.iterrows():

    current_linear_combination = lreg.intercept_[0]
    for attribute in list(coefs_map.keys()):
        attr_value = irow[attribute]
        attr_weighted = attr_value * coefs_map.get(attribute)
        current_linear_combination += attr_weighted
    
    prob_p1_win = round(exp(current_linear_combination) / (1 + exp(current_linear_combination)), 2)
    prob_p0_win = round(1 - prob_p1_win, 2)

    betting_df_list.append(   {"Pl0":irow['Player 0'], "Pl1":irow['Player 1'], "Winner":irow['Won'], "Pl0 %":prob_p0_win, "Pl1 %":prob_p1_win, \
                              "Pl0 Avg odds": irow['Avg_Pl0'], "Pl1 Avg odds": irow['Avg_Pl1'], "Pl0 Max odds": irow['Max_Pl0'],
                               "Pl1 Max odds": irow['Max_Pl1']
                              })
    
betting_df = pd.DataFrame(betting_df_list)

In [11]:
betting_df.head()

,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds
0,De Bakker T.,Falla A.,0.0,0.60,0.40,1.52,2.46,1.62,2.79
1,Starace P.,Hajek J.,1.0,0.66,0.34,1.34,3.12,1.40,3.64
2,Schwank E.,Fognini F.,0.0,0.35,0.65,1.99,1.77,2.14,1.85
3,Rochus C.,Garcia-Lopez G.,1.0,0.31,0.69,3.02,1.36,3.25,1.45
4,Bellucci T.,Nieminen J.,0.0,0.57,0.43,1.46,2.63,1.50,3.09


Let's define a helper funxtion that takes a change in input and returns the 'fair' odds for that chance value.

In [12]:
def right_odds(chance):
    return round(1/chance,2)

Now, we have everything we need to make decisions based on which player, for each match, we get the most value betting on!
In fact, next we're going to append to this betting dataframe another 4 columns.

The fair odds for both player 0 and player 1, based on the chances of each player to win according to the Logistic Regression model, and the value that we would get betting on each player at the max odds available!

In [13]:
for ix, row in betting_df.iterrows():
    right_odds_p0 = right_odds(row['Pl0 %'])
    right_odds_p1 = right_odds(row['Pl1 %'])
    bet_value_p0 = (row['Pl0 Max odds'] - right_odds_p0)/max(right_odds_p0,row['Pl0 Max odds'])
    bet_value_p1 = (row['Pl1 Max odds'] - right_odds_p1)/max(right_odds_p1,row['Pl1 Max odds'])
    betting_df.at[ix, 'Fair odds for Pl0'] = right_odds_p0
    betting_df.at[ix, 'Fair odds for Pl1'] = right_odds_p1
    betting_df.at[ix, 'Bet on Pl0 Value'] = bet_value_p0
    betting_df.at[ix, 'Bet on Pl1 Value'] = bet_value_p1

In [14]:
betting_df

,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds,Fair odds for Pl0,Fair odds for Pl1,Bet on Pl0 Value,Bet on Pl1 Value
0,De Bakker T.,Falla A.,0.0,0.60,0.40,1.52,2.46,1.62,2.79,1.67,2.50,-0.029940,0.103943
1,Starace P.,Hajek J.,1.0,0.66,0.34,1.34,3.12,1.40,3.64,1.52,2.94,-0.078947,0.192308
2,Schwank E.,Fognini F.,0.0,0.35,0.65,1.99,1.77,2.14,1.85,2.86,1.54,-0.251748,0.167568
3,Rochus C.,Garcia-Lopez G.,1.0,0.31,0.69,3.02,1.36,3.25,1.45,3.23,1.45,0.006154,0.000000
4,Bellucci T.,Nieminen J.,0.0,0.57,0.43,1.46,2.63,1.50,3.09,1.75,2.33,-0.142857,0.245955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17099,Zverev A.,Youzhny M.,0.0,0.54,0.46,1.60,2.30,1.66,2.45,1.85,2.17,-0.102703,0.114286
17100,Troicki V.,Wawrinka S.,1.0,0.17,0.83,4.34,1.21,4.91,1.24,5.88,1.20,-0.164966,0.032258
17101,Zverev A.,Berdych T.,0.0,0.16,0.84,2.68,1.46,3.12,1.57,6.25,1.19,-0.500800,0.242038
17102,Bautista Agut R.,Wawrinka S.,1.0,0.27,0.73,3.40,1.31,3.80,1.41,3.70,1.37,0.026316,0.028369


We just obtained 17104 matches where we can bet on the player who's value is higher!
Let's save this dataframe!

In [15]:
betting_df.to_csv("csv/Betting_Data.csv", index=False)